# Experimento 2: ruido microsísmico 

En [Processing seismic ambient noise data to obtain reliable broad-band
surface wave dispersion measurements](http://onlinelibrary.wiley.com/doi/10.1111/j.1365-246X.2007.03374.x/abstract), Bensen et al presentan una metodología para hacer inversiones a partir de ruido microsísmico. En este experimento vamos a transformar en código parte de esta metodología, para demostrar que programar es solo una herramienta más en el trabajo de invertigación. 

La metodología se puede resumir así: 

- Tome un día de señal de un par de estaciones. 
- Aplique el preprocesamiento a ambas: de-trend, de-mean, remove_response, filter...
- Aplique un «moving average». 
- Calcule la cross-correlación entre ambas señales. 
- Sume la cross-correlación. 

Primero escriba una función que tome el nombre de un archivo, lo abra y preprocese la señal. Retorne solamente `Trace.data`, no el `Stream`. La banda del ruido microsísmico se encuentra entre 7s y 150s, utilice entonces un filtro pasa-banda. 

In [6]:
import obspy

def pre_process(signal_name, resp):
    signal = obspy.read(signal_name)
    signal.detrend(type='linear')
    signal.remove_response(resp, output='DISP', water_level=0)
    signal.filter('bandpass', freqmin=1/150, freqmax=1/7)
    return signal[0]

Para al *moving average*, Bensen et al. recomienda utilizar la mitad del máximo periodo del filtro pasabanda, es decir, 75s. La señal se muestrea a 100 Hz, esto significa que la ventana será $N = 75 \cdot 100 = 7500$. Para calcular esta operación, utilizaremos la función `cumsum` de Numpy, que calcula la suma acumulativa de un areglo:

In [4]:
import numpy as np

def moving_average(signal, N):
    cumsum = np.cumsum(abs(signal))
    signal_mean = (cumsum[N:] - cumsum[:-N]) / float(N)
    return signal[N/2:-N/2] / signal_mean

Ahora ya tenemos todo lo necesario para estimar la función de Green con ruido microsísmico. Revise el directorio *Home*, ahí debe encontrar un directorio llamado `ruido_microsismico`, donde se encuentra todo el mes de Marzo 2015 de las estaciones VTUN y VTCV (Crater central y Calle Vargas) de la red del OVSICORI. El número de los archivos va desde 0 hasta el 29. 

Para simplificar el código, escriba una función que tome el nombre de un archivo, se lo pase a `pre_process` y seguidamente calcule el *moving average*. Retorne este arreglo. 


In [2]:
def pre_noise(trace, resp, N):
    signal = pre_process(trace, resp)
    return moving_average(array, N)

Con esta función, abra todos los pares de días de ambas estaciones, y calcule la cross-correlación. Sume la señal al arreglo `green`. Note que solo nos interesa la parte media del arreglo, unos 2000 s alrededor de la mitad.

In [13]:
import matplotlib.pyplot as plt 

# tamaño del arreglo para la función de green 2000s a 100 Hz
w = 2000 * 100
green = np.zeros(w)

N = 7500 # Tamaño de la ventana para el moving_average 

# Inventario para eliminar la respuesta del instrumento. 
inv = obspy.read_inventory('/home/gmocornejos/ruido_microsismico/OV_resp_all.dataless')

for i in range(1):
    signal1 = pre_noise('/home/gmocornejos/ruido_microsismico/Turrialba-Calle_Vargas_%d.tar.mseed'%i, inv, N)
    signal2 = pre_noise('/home/gmocornejos/ruido_microsismico/Turrialba-Crater_central_%d.tar.mseed'%i, inv, N)
    cross = numpy.correlate(signal1, signal2, mode='same')[source]
    mid = int(len(cross)/2)
    green += cross[mid-w/2:mid+w/2]

TypeError: filter() takes 2 positional arguments but 4 were given